# Preprocessing #2

In [123]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, RobustScaler, MaxAbsScaler, Normalizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

from sklearn import set_config
set_config(transform_output='pandas')

from category_encoders import OrdinalEncoder

from src.helpers import *


In [124]:
df = pd.read_csv('../data/train.csv')

In [125]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [126]:
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()

In [127]:
numeric_columns = df.select_dtypes(include=['int64']).columns.tolist()
numeric_columns = pd.DataFrame(numeric_columns)
numeric_columns = numeric_columns.values.flatten().tolist()
numeric_columns.remove('SalePrice')

In [128]:
scaler = StandardScaler()

In [129]:
preprocessor = ColumnTransformer([
  ('scaler',scaler,numeric_columns)
], remainder='drop', verbose_feature_names_out=False)

In [130]:
X = df.drop(columns='SalePrice')
y = df['SalePrice'].copy()

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [133]:
from sklearn.ensemble import RandomForestRegressor

main_pipe = Pipeline([
  ('preprocessor',preprocessor)
])

### Random Forest Regressor

In [137]:
rf = RandomForestRegressor()

In [139]:
rf.fit(X_train, y_train)

X_train_proc = main_pipe.transform(X_train)
X_val_proc = main_pipe.transform(X_val)

linear_results_train = evaluate_regression(rf, X_train_proc, y_train, 'rf')
linear_results_val = evaluate_regression(rf, X_val_proc, y_val, 'rf')

ValueError: could not convert string to float: 'RL'

In [ ]:
linear_results_train

,MAE,MSE,RMSE,MAPE,R2,adj_r2
rf,6718.388142,1.330650e+08,11535.379812,0.039137,0.977691,0.977021


In [ ]:
linear_results_val

,MAE,MSE,RMSE,MAPE,R2,adj_r2
rf,17919.430788,8.121708e+08,28498.610382,0.108935,0.894115,0.880107


In [ ]:
val_preds = rf.predict(X_val_proc)
pred_log = np.log(val_preds)
y_log = np.log(y_val)
root_mean_squared_error(y_log, pred_log)

0.15594603817592861

In [ ]:
df_test = pd.read_csv('../data/test.csv')

In [ ]:
df_test_pred = rf.predict(df_test)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- Alley
- BldgType
- BsmtCond
- BsmtExposure
- BsmtFinType1
- ...
